In [11]:
#Fine-Grained Feature Squeeze Module
def FGFS(Skip_Connecton_Input,Upsample_Input):
    
    encoder_channels=Skip_Connecton_Input.shape[-1]

    Skip_Connecton_Input_1=Conv2D(encoder_channels, kernel_size=1, padding='same')(Skip_Connecton_Input)
    Skip_Connecton_Input_2=Conv2D(encoder_channels//4, kernel_size=3, padding='same')(Skip_Connecton_Input_1)
    Skip_Connecton_Input_3=Conv2D(encoder_channels, kernel_size=1, padding='same')(Skip_Connecton_Input_2)
    Upsample_Input_1=Conv2D(encoder_channels, kernel_size=1, padding='same',dilation_rate=1)(Upsample_Input)
    Upsample_Input_2=Conv2D(encoder_channels//4, kernel_size=3, padding='same',dilation_rate=3)(Upsample_Input_1)
    Upsample_Input_3=Conv2D(encoder_channels, kernel_size=1, padding='same',dilation_rate=5)(Upsample_Input_2)
    att = tf.sigmoid(Upsample_Input_3)
    mul=tf.multiply(Skip_Connecton_Input_3, att)
    encoder_att = Add()([Skip_Connecton_Input,mul])
    cat = concatenate([encoder_att, Upsample_Input_3], axis=3)
    out=Conv2D(encoder_channels, kernel_size=1, padding='valid')(cat)

    return out


(None, 16, 16, 128)


In [9]:
#Multiscale Semantic Aggregation Module
def MSA(input_1,input_2):
        channel_1=input_1.shape[-1]
        channel_2=input_2.shape[-1]
        # channel_3=input_3.shape[-1]
        # channel_4=input_4.shape[-1]
        # channel_5=input_5.shape[-1]
        
        input_1_1_1 =AveragePooling2D(pool_size=(2, 2))(input_1)
        input_1_2 = tf.concat([input_1_1_1, input_2], axis=3)
        input_1_2 = Conv2D(channel_2, kernel_size=(1, 1), padding='same')(input_1_2)

        return input_1_2
#Multiscale Dilated Semantic Aggregation Module
def MDSA(input_1,input_2, input_3):
        channel_1=input_1.shape[-1]
        channel_2=input_2.shape[-1]
        channel_3=input_3.shape[-1]
        # channel_4=input_4.shape[-1]
        # channel_5=input_5.shape[-1]
        
        input_1_1_1 =AveragePooling2D(pool_size=(2, 2))(input_1)
        input_1_2 = tf.concat([input_1_1_1, input_2], axis=3)
        input_1_2 = Conv2D(channel_2, kernel_size=(1, 1), padding='same', dilation_rate=1)(input_1_2)
        
        input_1_2_2_2 =AveragePooling2D(pool_size=(2, 2))(input_2)
        input_1_2_3 = tf.concat([input_1_2_2_2, input_3], axis=3)
        input_1_2_3 = Conv2D(channel_3, kernel_size=(3, 3), padding='same')(input_1_2_3)
        

        return input_1_2_3
#Multiscale Cascade Dilated Semantic Aggregation Module
def MCDSA(input_1,input_2, input_3, input_4):
        channel_1=input_1.shape[-1]
        channel_2=input_2.shape[-1]
        channel_3=input_3.shape[-1]
        channel_4=input_4.shape[-1]
        
        input_1_1_1 =AveragePooling2D(pool_size=(2, 2))(input_1)
        input_1_2 = tf.concat([input_1_1_1, input_2], axis=3)
        input_1_2 = Conv2D(channel_2, kernel_size=(1, 1), padding='same', dilation_rate=1)(input_1_2)
        
        input_1_2_2_2 =AveragePooling2D(pool_size=(2, 2))(input_2)
        input_1_2_3 = tf.concat([input_1_2_2_2, input_3], axis=3)
        input_1_2_3 = Conv2D(channel_3, kernel_size=(3, 3), padding='same', dilation_rate=2)(input_1_2_3)
        
        
        input_1_2_3_3_3 =AveragePooling2D(pool_size=(2, 2))(input_1_2_3)
        input_1_2_3_4 = tf.concat([input_1_2_3_3_3, input_4], axis=3)
        input_1_2_3_4 = Conv2D(channel_4, kernel_size=3, padding='same')(input_1_2_3_4)
        

        return input_1_2_3_4


In [18]:
#FMD-UNet
from tensorflow.keras import backend as K
import tensorflow as tf
K.clear_session()
IMG_HEIGHT=128
IMG_WIDTH=128
IMG_CHANNELS=1
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
#Contraction path
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
# c1 = Dropout(0.1)(c1)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)
#
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
# c2 = Dropout(0.1)(c2)
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = MaxPooling2D((2, 2))(c2)
#
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
# c3 = Dropout(0.2)(c3)
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = MaxPooling2D((2, 2))(c3)
#
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
# c4 = Dropout(0.2)(c4)
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = MaxPooling2D(pool_size=(2, 2))(c4)
#
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
# c5 = Dropout(0.3)(c5)
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

MSA_OUTPUT=MSA(c1,c2)
MDSA_OUPUT=MDSA(c1,c2,c3)
MCDSA_OUTPUT=MCDSA(c1,c2,c3,c4)

# decoder_branch one
u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
FGFS_one = FGFS(u6,c4)
u6 = concatenate([FGFS_one,u6, c4])
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
# c6 = Dropout(0.2)(c6)
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

# c6_sup=Conv2D(1, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
# c6_sup=cv.resize(arr_cts[ii], dsize=(img_size, img_size), interpolation=cv.INTER_AREA)
#
u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
FGFS_two = FGFS(u7,c3)
u7 = concatenate([FGFS_two,u7, c3])
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
# c7 = Dropout(0.2)(c7)
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

# c7_sup=Conv2D(1, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
# 
u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
FGFS_three = FGFS(u8,c2)
u8 = concatenate([FGFS_three,u8, c2])
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
# c8 = Dropout(0.1)(c8)
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

# c8_sup=Conv2D(1, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
FGFS_four=FGFS(u9,c1)
u9 = concatenate([FGFS_four,u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
# c9 = Dropout(0.1)(c9)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

#deepsuperviso
c6_sup_1=UpSampling2D(size=(8, 8))(c6)
c7_sup_1=UpSampling2D(size=(4, 4))(c7)
c8_sup_1=UpSampling2D(size=(2, 2))(c8)
c9_sup_1=concatenate([c6_sup_1,c7_sup_1,c8_sup_1,c9])
c9_sup_1=Conv2D(1, (1, 1), activation='sigmoid',name='c9_sup_1')(c9_sup_1)

#=======================decoder_branch two====================
u6_1= Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6_1= concatenate([u6_1, MCDSA_OUTPUT])
c6_1= Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6_1)
# c6_1 = Dropout(0.2)(c6_1)
c6_1 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6_1)
#
u7_1 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6_1)
u7_1 = concatenate([u7_1, MDSA_OUPUT])
c7_1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7_1)
# c7_1 = Dropout(0.2)(c7_1)
c7_1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7_1)
#
u8_1 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7_1)
u8_1 = concatenate([u8_1, MSA_OUTPUT])
c8_1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8_1)
# c8_1 = Dropout(0.1)(c8_1)
c8_1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8_1)
#
u9_1 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8_1)
u9_1 = concatenate([u9_1, c1], axis=3)
c9_1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9_1)
# c9_1 = Dropout(0.1)(c9_1)
c9_1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9_1)

# c9_sup_2=Conv2D(1, (1, 1), activation='sigmoid')(c9_1)
#
all_output=concatenate([c9,c9_1],axis=3)
#
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid',name='outputs')(all_output)
losses={
    "c9_sup_1":weighted_bce_dice_loss,
    "outputs":weighted_bce_dice_loss
}
loss_weight={
    "c9_sup_1":0.3,
    "outputs":0.7
}
#
model = tf.keras.Model(inputs=inputs, outputs=[outputs,c9_sup_1])
# model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

c1.shape(None, 128, 128, 16),c2.shape(None, 64, 64, 32),c3.shape(None, 32, 32, 64),c4.shape(None, 16, 16, 128)
c2_2(None, 64, 64, 32)
c3_2(None, 64, 64, 32)
c4_2(None, 64, 64, 32)
=====================u6(None, 16, 16, 128)========c4(None, 16, 16, 128)
wgm_one.shape(None, 16, 16, 128)
=====================u7(None, 32, 32, 64)========cC(None, 32, 32, 64)
wgm_TWO.shape(None, 32, 32, 64)
(None, 128, 128, 128)
(None, 128, 128, 64)
(None, 128, 128, 32)
(None, 128, 128, 240)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda) 